In [1]:
import copy;#permite guardar un objeto
# Qué es un estado
# 'Set' significa conjunto (cosas no repetidas), podríamos decir que 'set' es la orilla ennuestro problema
estado_inicial = [{'P', 'L', 'C', 'H'}, set()]
estado_final = [set(), {'P', 'L', 'C', 'H'}]

# 'S' significa estado
def is_valid_state(s):
    assert type(s)==list and len(s)==2;#Comprobamos que sea una lista y que dicha lista tenga 2 elementos
    assert type(s[0])==set;#orilla izquierda
    assert type(s[1])==set;#basicamente que tiene que haber un conjunto tanto en el primer termino como en el segundo del estado 
    
    #Esta 's' no tiene nada que ver con la de arriba y significa shore
    def is_valid_shore(s):
        assert s <= {'P','L','C','H'}
        if {'P'}      <= s: return True#si el paisano está en la orilla
        if {'L', 'C'} <= s: return False#si el cordero y el lobo están juntos
        if {'C', 'H'} <= s: return False#si el cordero y el heno están juntos
        return True;
    
    if is_valid_shore(s[0]) and is_valid_shore(s[1]): return True
    else:                                             return False
    #comprueba en la función si la orilla de la derecha e izquierda son validas,si es así es verdadero, si no, falsa

# esta s no tiene que ver con las demás 
def states(s):
    assert type(s)==list and len(s)==2;#Comprobamos que sea una lista y que dicha lista tenga 2 elementos
    assert type(s[0])==set;#orilla derecha
    assert type(s[1])==set;#basicamente que tiene que haber un conjunto tanto en el primer termino como en el segundo del estado
    
    #lp (lado del paisano)
    lp=0 if 'P' in s[0] else 1;#Lado en el que está el paisano
    #np (no está el paisano)
    np=1 if 'P' in s[0] else 0;#Lado en el que NO está el paisano
    #ns (nuevo estado)
    ns=copy.deepcopy(s)#copia profunda de 's'
    ns[np].add('P')#añadimos al paisano en el conjunto np (donde no está el paisano)
    ns[lp].remove('P')#quitamos el paisano al conjunto lp (donde SI está el paisano)
    yield ns;#nos devuelve ns, vuelve desde donde lo dejó
    
    for c in s[lp].difference({'P'}):#va d elemento en elemento en ese conjunto y hace los estados posibles (c es uno de los elementos de la lista que contiene 'C','H','L')
        ns=copy.deepcopy(s)
        ns[np].add('P')
        ns[np].add(c)
        ns[lp].remove('P')
        ns[lp].remove(c)
        yield ns


for s in states(estado_inicial):#imprime todas las combinaciones posibles (pero primero el estado inicial)
    print(s)
    

[{'C', 'L', 'H'}, {'P'}]
[{'L', 'H'}, {'C', 'P'}]
[{'C', 'H'}, {'P', 'L'}]
[{'C', 'L'}, {'H', 'P'}]


In [2]:
abiertos=[estado_inicial]
cerrados=[]

actual=abiertos[0]
while actual!=estado_final and len(abiertos)>0:#mientras actual no sea el estado final y en abiertos haya algo
    abiertos.pop(0)#quita ese estado(en este caso el primero por eso es '0')
    cerrados.append(actual)#el estado actual se añade a estados cerrados 
    for ns in states(actual):#ns es nuevo estado (va a recorrer a sus nuevos hijos)
        if is_valid_state(ns):#si el estado es valido
            abiertos.append(ns)#significa que habrá que vistar ese estado
    
    actual=abiertos[0]
    
if actual==estado_final:
    print("Hay solución")
    
else:
    print("No hay solución")

Hay solución


In [3]:
#Función recursiva (el camino para llegar a la solución) Primera solución
def búsquedaProfundidad(actual, cerrados=[]):
    #En una función recursiva es importante saber que casos no son posibles
    if actual==estado_final:
        return [actual]
    else:
        for ns in states(actual):#se van generando los hijos
            if is_valid_state(ns) and ns not in cerrados:#si es valido el estado (el hijo) y si NO LO HE VISITADO
                cerrados.append(ns)
                r=búsquedaProfundidad(ns, cerrados)#cuando llega al estado final; vuelve desde el estado final por los hijos (o estados) que atravesó para llegar a estado final 
                if r!=[]:#si r es diferente a vacio (osea que el estado es válido)
                    r.insert(0, actual)#por lo que si encuentra un estado valido lo que hace es que, recursivamente vuelve y va marcando cuales son los estados con los que llegó a esa solución; ejemplo(si el estado final es [6] va hacer lo siguiente:1)[5,6]; 2)[4,6] y así hasta llegar [0,6])
                    
                    return r
        return[]

solución = búsquedaProfundidad(estado_inicial)
solución

[[{'C', 'H', 'L', 'P'}, set()],
 [{'H', 'L'}, {'C', 'P'}],
 [{'H', 'L', 'P'}, {'C'}],
 [{'L'}, {'C', 'H', 'P'}],
 [{'C', 'L', 'P'}, {'H'}],
 [{'C'}, {'H', 'L', 'P'}],
 [{'C', 'P'}, {'H', 'L'}],
 [set(), {'C', 'H', 'L', 'P'}]]

In [4]:
#Función recursiva (el camino para llegar a la solución)BUSQUEDA DE TODAS LAS SOLUCIONES
def búsquedaProfundidadPlus(actual, path=[]):
    #En una función recursiva es importante saber que casos no son posibles
    if actual==estado_final:
        print("Solución:")#imprime la solución y sigue
        for s in path: #imprime el los hijos donde va consiguiendo solución
            print("\t",s)
        print("\t",estado_final)#imprime el estado final al que llega

    else:
        path.append(actual)#se añade a si misma a la derecha
        for ns in states(actual):#se van generando los hijos de actual
            if is_valid_state(ns) and ns not in path:#si es valido el estado (el hijo) y no se genera un ciclo (no está en el path)
                búsquedaProfundidadPlus(ns,path)
        path.pop()
        

solución = búsquedaProfundidadPlus(estado_inicial)
solución

Solución:
	 [{'C', 'P', 'L', 'H'}, set()]
	 [{'L', 'H'}, {'C', 'P'}]
	 [{'H', 'P', 'L'}, {'C'}]
	 [{'L'}, {'C', 'P', 'H'}]
	 [{'C', 'P', 'L'}, {'H'}]
	 [{'C'}, {'H', 'P', 'L'}]
	 [{'C', 'P'}, {'H', 'L'}]
	 [set(), {'C', 'P', 'L', 'H'}]
Solución:
	 [{'C', 'P', 'L', 'H'}, set()]
	 [{'L', 'H'}, {'C', 'P'}]
	 [{'H', 'P', 'L'}, {'C'}]
	 [{'H'}, {'C', 'P', 'L'}]
	 [{'H', 'P', 'C'}, {'L'}]
	 [{'C'}, {'H', 'P', 'L'}]
	 [{'C', 'P'}, {'H', 'L'}]
	 [set(), {'C', 'P', 'L', 'H'}]


In [5]:
#Función con yield
def búsquedaProfundidadPlus(actual, path=[]):
    #En una función recursiva es importante saber que casos no son posibles
    if actual==estado_final:
        yield path + [actual]#devuelve path más el estado actual

    else:
        path.append(actual)#se añade a si misma a la derecha
        for ns in states(actual):#se van generando los hijos de actual
            if is_valid_state(ns) and ns not in path:#si es valido el estado (el hijo) y no se genera un ciclo 
                yield from búsquedaProfundidadPlus(ns,path)#el yield from indica de donde va a hacer el yield 
        path.pop()
        
for solución in búsquedaProfundidadPlus(estado_inicial):
    print("\n Solución",solución)


 Solución [[{'C', 'P', 'L', 'H'}, set()], [{'L', 'H'}, {'C', 'P'}], [{'H', 'P', 'L'}, {'C'}], [{'L'}, {'C', 'P', 'H'}], [{'C', 'P', 'L'}, {'H'}], [{'C'}, {'H', 'P', 'L'}], [{'C', 'P'}, {'H', 'L'}], [set(), {'H', 'P', 'L', 'C'}]]

 Solución [[{'C', 'P', 'L', 'H'}, set()], [{'L', 'H'}, {'C', 'P'}], [{'H', 'P', 'L'}, {'C'}], [{'H'}, {'C', 'P', 'L'}], [{'H', 'P', 'C'}, {'L'}], [{'C'}, {'H', 'P', 'L'}], [{'C', 'P'}, {'H', 'L'}], [set(), {'H', 'P', 'L', 'C'}]]
